In [1]:
import pandas as pd
path = "/content/sample.csv"
df=pd.read_csv(path)

In [2]:
df.head()

,producto,presentacion,marca,categoria,catalogo,precio,fechaRegistro,cadenaComercial,giro,nombreComercial,direccion,estado,municipio,latitud,longitud
0,CUADERNO FORMA ITALIANA,96 HOJAS PASTA DURA. CUADRICULA CHICA,ESTRELLA,MATERIAL ESCOLAR,UTILES ESCOLARES,25.9,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
1,CRAYONES,CAJA 12 CERAS. JUMBO. C.B. 201423,CRAYOLA,MATERIAL ESCOLAR,UTILES ESCOLARES,27.5,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
2,CRAYONES,CAJA 12 CERAS. TAMANO REGULAR C.B. 201034,CRAYOLA,MATERIAL ESCOLAR,UTILES ESCOLARES,13.9,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
3,COLORES DE MADERA,CAJA 12 PIEZAS LARGO. TRIANGULAR. C.B. 640646,PINCELIN,MATERIAL ESCOLAR,UTILES ESCOLARES,46.9,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
4,COLOR LARGO,CAJA 36 PIEZAS. CON SACAPUNTAS. 68-4036,CRAYOLA,MATERIAL ESCOLAR,UTILES ESCOLARES,115.0,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417


##1. How Many commercial chains are monitored, and therefore, included in this database?

In [3]:
df.cadenaComercial.value_counts(dropna = False)
#No missing values

COMERCIAL MEXICANA    1508
WAL-MART               287
I.S.S.S.T.E.           195
ABASTECEDORA LUMEN       9
Name: cadenaComercial, dtype: int64

##2. What are the top 10 monitored products by State?

In [4]:
df.groupby("estado").apply(lambda x: x.producto.value_counts(dropna = False).nlargest(10))

estado                                   
DISTRITO FEDERAL  DETERGENTE P/ROPA          25
                  LECHE ULTRAPASTEURIZADA    23
                  REFRESCO                   22
                  CHILES EN LATA             21
                  MAYONESA                   20
                  SHAMPOO                    18
                  TELEVISORES                17
                  JABON DE TOCADOR           16
                  GALLETAS DULCES            16
                  MARGARINA                  14
MÉXICO            REFRESCO                   26
                  DETERGENTE P/ROPA          24
                  SHAMPOO                    19
                  CHILES EN LATA             18
                  TOALLA FEMENINA            17
                  MAYONESA                   17
                  LECHE ULTRAPASTEURIZADA    17
                  YOGHURT                    15
                  TELEVISORES                15
                  DESODORANTE                1

##3. Which is the commercial chain with the highest number of monitored products?

In [5]:
df.groupby("cadenaComercial").producto.nunique().sort_values(ascending=False).head(1)

cadenaComercial
COMERCIAL MEXICANA    310
Name: producto, dtype: int64

##4. Use the data to find an interesting fact.

In [6]:
expensive = df.precio.quantile(0.95)
e_df = df[df.precio > expensive] 
expensive

1224.9999999999845

In [7]:
#All products
df.groupby("estado").municipio.value_counts(normalize = True)

estado            municipio                               
DISTRITO FEDERAL  BENITO JUAREZ                               0.783439
                  IZTACALCO                                   0.207006
                  TLALPAN                                     0.009554
MÉXICO            ATIZAPAN                                    0.728477
                  NAUCALPAN                                   0.271523
Name: municipio, dtype: float64

In [8]:
#Only expensive products
e_df.groupby("estado").municipio.value_counts(normalize = True)

estado            municipio                               
DISTRITO FEDERAL  BENITO JUAREZ                               1.000000
MÉXICO            ATIZAPAN                                    0.824561
                  NAUCALPAN                                   0.175439
Name: municipio, dtype: float64

#### Where are expensive products located?
#### Mexico expensive products are balanced between cities. Distrito Federal expensive products are found only in Benito Juarez.

## 5. What are the lessons learned from this exercise?

####Not many, I was already familiar with Pandas.

## 6. Can you identify other ways to approach this problem? Explain.

#### I do not have enough RAM to load the complete file(20 GB), to make a complete analysis we would need to make process it in a distributed manner. Downloading the file to databricks and then running the analysis using PySpark APIs would allows to handle the whole file.

#### We can even use the [Koalas](https://koalas.readthedocs.io/en/latest/) library to reuse the pandas code on spark altough it would run slower than using only Spark APIs.


